In [ ]:
import sys
import logging
from os import walk
from requests import get,post,delete,put
from mimetypes import guess_type
from urllib.parse import quote
from time import sleep
from csv import DictReader,DictWriter
from re import sub
from math import ceil

In [ ]:
class pypeertube(object):
    """Batch uploads videos to a peertube instance"""
    def __init__(self):
        self.files_list = list()
        self.peertube_files_list = list()
        from settings import auth_data,config,custom_tags
        self.config = config
        if self.config["path"][-1:] == "/":
            self.config["path"] = self.config["path"][:-1]
        self.file_extensions = [".flv",
                                ".mp4",
                                ".wmv",
                                ".webm",
                                ".mkv",
                                ".3gpp",
                                ".mov",
                                ".asf",
                                ".mpeg"]

        self.custom_tags = custom_tags
        self.auth_data = auth_data
        self.auth_data["grant_type"] = "password"
        self.auth_data["response_type"] = "code"
        
        
        super(pypeertube, self).__init__()
        
        logging.basicConfig(
                            filename="event.log",
                            level=logging.INFO,
                            format='%(asctime)s %(levelname)s %(processName)s %(message)s', 
                            datefmt='%d.%m.%Y %H:%M:%S %p'
                            ) 
        
        if not self.config["channelID"]:
            self.config["channelID"] = get('{0}{1}/{2}'.format(self.config["site"], 
                                                                        '/api/v1/video-channels', 
                                                                        config["channel"]
                                                                       )).json()['id']
            logging.info("channelID: {}".format(self.config["channelID"]))
    
        self.upload_url = self.config["site"] + "/api/v1/videos/upload"
        logging.info("upload_url: {}".format(self.upload_url))

        self.csv_file = "files_list.csv"
        self.csv_file2 = "peertube_files_list.csv"
        logging.info("csv_file: {}".format(self.csv_file))

        self.file_mimetype = "video/mp4"
        logging.info("file_mimetype: {}".format(self.file_mimetype))
        
        self.oauth_url = config["site"] + "/api/v1/oauth-clients/local"
        logging.info("oauth_url: {}".format(self.oauth_url))

        self.auth_data["client_id"] = get(self.oauth_url).json()["client_id"]
        logging.info("client_id: {}".format(self.auth_data["client_id"]))

        self.auth_data["client_secret"] = get(self.oauth_url).json()["client_secret"]
        logging.info("client_secret: {}".format(self.auth_data["client_secret"]))

        self.token_url = self.config["site"] + "/api/v1/users/token"
        logging.info("token_url: {}".format(self.token_url))

        self.token_data = post(self.token_url, data=self.auth_data).json() #access_token,token_type,expires_in,refresh_token
        logging.info(str(self.token_data))
        logging.info("access_token: {}".format(self.token_data["access_token"]))
        logging.info("token_type: {}".format(self.token_data["token_type"]))
        logging.info("expires_in: {}".format(self.token_data["expires_in"]))
        logging.info("refresh_token: {}".format(self.token_data["refresh_token"]))
        
        self.upload_headers = {'Authorization': 'Bearer {0}'.format(self.token_data["access_token"])}
        
        self.upload_data = {
                'channelId': self.config["channelID"],
                'privacy': self.config["privacy"],
                'commentsEnabled': self.config["commentsEnabled"],
                #'name': filename
               }
    def add_custom_tags(self):
        
        self.check_csv_tags()
        
        for i in range(len(self.files_list)):
            title = self.files_list[i]["title"]
            tags = self.files_list[i]["tags"]

            for tag in self.custom_tags:
                if tag.lower() in title.lower():
                    if tag not in tags:
                        if len(tags) < 5:
                            tags.append(tag)

            if len(tags) == 1:
                tags.append("None")
            self.files_list[i]["tags"] = tags
        
        self.write_csv(self.csv_file,self.files_list)
    
    
    def get_all_tags(self):
        #returns tags of all videos as unique list
        pass
    
    def replace_tag(old_tag,new_tag):
        for i in range(len(self.peertube_files_list)):
            tags = self.peertube_files_list[i]["tags"]
            name = self.peertube_files_list[i]["name"]
            for index in range(len(tags)):
                if old_tag in tags[index]:
                    tags[index] = new_tag
    
    def index_peertube_videos(self):
        total_number_of_videos = self.get_videos(0,0).json()["total"]
        max_cycles = ceil(total_number_of_videos / 100)
        self.peertube_files_list = list()
        count = 100
        start = 1
        delay = 0.2
        
        for cycle in range(max_cycles):
            logging.info("----- Reading data cycle {} of {} -----".format(cycle+1,max_cycles))

            res = self.get_videos(count,start)

            for video in res.json()["data"]:
                sleep(delay)
                uuid = video["uuid"]
                metadata = self.get_video_metadata(uuid)
                logging.info("{} - {}".format(metadata["name"],metadata["state"]["label"]))
                self.peertube_files_list.append(metadata)
                
            logging.info("----- {} of {} videos read -----".format(len(self.peertube_files_list),total_number_of_videos))
            
            start += 100
            
        self.write_csv(self.csv_file2,self.peertube_files_list)
      
    
    def create_list_of_video_files(self):
        self.files_list = list()
        for root, subdirs, files in walk(self.config["path"]):            
            for file in files:
                skippme = False
                if file not in self.files_list:
                    file_location = root+"/"+file.replace(";",",")
                    file_mimetype = guess_type(file_location)[0]
                    filename = file.replace(";",",")
                    title = self.delete_file_extensions_from_videoname(filename)
                    if isinstance(self.config["category"], str):
                        category = self.config["category"].replace(";","")
                    if isinstance(self.config["licence"], str):
                        category = self.config["licence"].replace(";","")
                    if isinstance(self.config["language"], str):
                        category = self.config["language"].replace(";","")
                    if isinstance(self.config["nsfw"], str):
                        category = self.config["nfsw"].replace(";","")
                    if isinstance(self.config["commentsEnabled"], str):
                        category = self.config["commentsEnabled"].replace(";","")

                    #'file_mimetype': 'video/x-flv'}
                    #'file_mimetype': 'video/mp4'}
                    #'file_mimetype': 'video/x-ms-wmv'}
                    #'file_mimetype': 'video/webm'}
                    #'file_mimetype': 'video/x-msvideo'}
                    #'file_mimetype': 'video/x-matroska'}
                    #'file_mimetype': 'video/3gpp'}
                    #'file_mimetype': 'video/quicktime'}
                    #'file_mimetype': 'video/x-ms-asf'}
                    #'file_mimetype': 'video/mpeg'}
                    
                    if file_mimetype:
                        if "video" in file_mimetype:
                            for exclude in self.config["exclude_list"]:
                                exclude = exclude
                                if exclude != "":
                                    if exclude in file_location:
                                        logging.info("Excluding \"{}\" from uploading".format(file_location))
                                        skippme = True
                            if not skippme:
                                self.files_list.append({
                                       "title":title,
                                       "filename":filename,
                                       "file_location":file_location,
                                       "file_mimetype":file_mimetype,
                                       "category":self.config["category"],
                                       "licence":self.config["licence"],
                                       "language":self.config["language"],
                                       "nsfw":self.config["nsfw"],
                                       "commentsEnabled":self.config["commentsEnabled"],
                                       "tags":None,
                                       "status":"queued"
                                                  })
        self.write_csv(self.csv_file,self.files_list)
        if self.files_list:
            return self.files_list
    
    def load_csv(self):
        self.files_list = list()
        self.peertube_files_list = list()

        try:
            with open(self.csv_file) as csvfile:
                reader = DictReader(csvfile)
                for row in reader:
                    self.files_list.append(dict(row))
                    
            self.check_csv_tags()

        except Exception as e:
            logging.error(e)
            logging.info("Unable to import files_list.csv. Recreating files list from scratch")
            logging.info("Finding every videofile in \"{}\"".format(self.config["path"]))

            self.files_list = self.create_list_of_video_files()
            
            self.check_csv_tags()
            self.add_custom_tags()

            if self.files_list:
                logging.info("Found {} videos in \"{}\"".format(len(self.files_list),self.config["path"]))
                logging.info("Saved data in {}".format(self.csv_file))
                
        try:
            with open(self.csv_file2) as csvfile:
                reader = DictReader(csvfile)
                for row in reader:
                    self.peertube_files_list.append(dict(row))

        except Exception as e:
            logging.error("Error: {}".format(e))
            logging.info("Unable to import peertube_files_list.csv.")
            
        self.check_csv_tags()
        self.add_custom_tags()
        
    def write_csv(self,filename,data):
        keys = data[0].keys()
        with open(filename, 'w') as output_file:
            dict_writer = DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(data)
            
    def search(self,pattern):
        
        if isinstance(pattern,str):
            result_list = list()
            video_id_list = list()

            pattern_formatted = quote(pattern)
            pattern_cleaned_file_extension = quote(self.delete_file_extensions_from_videoname(pattern))
            sleep(0.5)
            search_url_pattern_formatted = self.config["site"] + "/api/v1/search/videos?start=0&count=10&search={}&sort=-match".format(pattern_formatted)
            search_url_pattern_cleaned_file_extension = self.config["site"] + "/api/v1/search/videos?start=0&count=10&search={}&sort=-match".format(pattern_cleaned_file_extension)
            
            try:
                #First round - try to find pattern
                results = get(search_url_pattern_formatted).json()["data"]
                for result in results:
                    if pattern == result["name"]:
                        if result["id"] not in video_id_list:
                            result_list.append(result)
                            video_id_list.append(result["id"])
                #Second round - try to find pattern without known file extentions
                results = get(search_url_pattern_cleaned_file_extension).json()["data"]
                for result in results:
                    if pattern == result["name"]:
                        if result["id"] not in video_id_list:
                            result_list.append(result)
                            video_id_list.append(result["id"])
                            
            except Exception as e:
                logging.error("unable to search")
                logging.error("Error: {}".format(e))
                
            finally:
                return result_list

    def sync(self):

        if not self.files_list:
            self.load_csv()

        logging.info("Syncing progress of files. Searching local filesnames/titels on the website")
        files_list_length = len(self.files_list)

        for i in range(files_list_length):
            sleep(0.2)
            video_title = self.files_list[i]["filename"]
            try:
                if isinstance(self.files_list[i]["tags"], str):
                    tags = eval(self.files_list[i]["tags"])
            except:
                logging.error("can not parse tags")
                logging.error("tags:{}".format(self.files_list[i]["tags"]))
                tags = list()
            search_result = None
            search_result = self.search(video_title)[0]
            if search_result == "error":
                self.files_list[i]["status"] = "error"
                logging.error("Video number {} of {} - error".format(i+1,files_list_length))
            else:
                if search_result:
                    logging.info("Video number {} of {} - completed".format(
                                                                    i+1,
                                                                    files_list_length
                                                                ))
                    self.files_list[i]["status"] = "completed"
                    uuid = search_result["uuid"]
                    
                    self.files_list[i]["uuid"] = uuid
                    self.set_video_metadata(uuid,{
                            "tags":tags,
                            "category":self.config["category"],
                            "licence":self.config["licence"],
                            "language":self.config["language"],
                            "privacy":self.config["privacy"],
                            "nsfw":self.config["nsfw"],
                            "commentsEnabled":self.config["commentsEnabled"],
                            "downloadEnabled":self.config["downloadEnabled"],
                            "support":self.config["support"]
                    })

                else:
                    logging.info("Video number {} of {} - not found".format(i+1,files_list_length))
        
        self.write_csv(self.csv_file,self.files_list)
        
        
    def delete_file_extensions_from_videoname(self,name):    
        for extension in self.file_extensions :
            if extension in name:
                return name.replace(extension, '')
        return name

    def get_video_metadata(self,video_uuid):
        url = "{}/api/v1/videos/{}".format(self.config["site"],video_uuid)
        res = get(url)
        
        return res.json()
    
    def delete_video(self,video_uuid):
        url = "{}/api/v1/videos/{}".format(self.config["site"],video_uuid)
        res = delete(
                              url,
                              headers=self.upload_headers
                             )
        return res

    def set_video_metadata(self,video_id,payload):
        url = "{}/api/v1/videos/{}".format(self.config["site"],video_id)
        return put(url,
                            headers = self.upload_headers,
                            data = payload
                           )

    def check_csv_tags(self):
        self.files_list_length = len(self.files_list)
        
        for i in range(self.files_list_length):
            tags = list()
            if self.files_list[i]["tags"]:
                if isinstance(self.files_list[i]["tags"],str):
                    imported_tags = eval(self.files_list[i]["tags"])
                elif isinstance(self.files_list[i]["tags"],list):
                    imported_tags = self.files_list[i]["tags"]

                for tag in imported_tags:
                    if len(tags) < 5:
                        if tag not in tags:
                            tags.append(tag)
                            
                tags = list(filter(None,tags))
                
                if len(tags) == 1:
                    tags.append("none")
                self.files_list[i]["tags"] = tags
            else:
                self.files_list[i]["tags"] = list()
                    
        self.write_csv(self.csv_file,self.files_list)
            

    def learn_tags_from_folder_structure_and_update_videos(self):
        
        self.files_list_length = len(self.files_list)

        for i in range(self.files_list_length):
            logging.info("Video number {} of {} has tags \"{}\"".format(i+1,
                                                                self.files_list_length,
                                                                self.files_list[i]["tags"]
                                                                       ))
            tags = list()
            if self.files_list[i]["tags"]:
                try:
                    imported_tags = eval(self.files_list[i]["tags"])
                    logging.info("imported_tags {} {}".format(
                                                            imported_tags,
                                                            type(imported_tags)
                                                        ))
                    if isinstance(imported_tags,list):
                        for tag in imported_tags:
                            if tag not in tags:
                                tags.append(tag)
                        tags = list(filter(None,tags))
                        self.files_list[i]["tags"] = tags

                        logging.info("tags {}".format(tags))
                except Exception as e: 
                    logging.error("unable to import tags")
                    logging.error("error: {}".format(e))
                    logging.error("imported tags \"{}\" type: \"{}\"".format(tags,
                                                                             type(tags)
                                                                            ))
        self.write_csv(self.csv_file,self.files_list)

    def get_videos(self,count,start):
        username = self.auth_data["username"]
        url = "{}/api/v1/accounts/{}/videos".format(self.config["site"],username)
        
        return get(
                            url,
                            params={'count': count, 'start': start}
                            )

    def string_cleanup(self,string):
        #removes special characters (including spaces) from string
        new_string = ""
        for char in string:
            if char == " ":
                new_string += char
            else:
                if char.isalpha():
                    new_string += char
        new_string = sub(r'\s+', ' ', new_string)
        return new_string

    
    def upload(self):
                
        self.files_list_length = len(self.files_list)
        for i in range(self.files_list_length):
            title = self.files_list[i]["title"]
            logging.info("-----------------------------")
            logging.info("Video {} of {} is uploading...".format(i+1,self.files_list_length))

            self.upload_data["name"] = self.delete_file_extensions_from_videoname(
                                                    self.files_list[i]["filename"]
                                                    )

            if self.files_list[i]["status"] == "completed" or self.files_list[i]["status"] == "error":
                logging.info("Skipping video \"{}\". It's already marked as completed or error.".format(
                                                        self.files_list[i]["filename"]
                                                        ))    
                
            else:
                logging.info(
                    "Uploading video \"{}\" from \"{}\" with mimetype \"{}\"".format(
                                                    self.files_list[i]["filename"],
                                                    self.files_list[i]["file_location"],
                                                    self.files_list[i]["file_mimetype"]
                                                    ))
                try:
                    self.upload_result = None
                    with open(self.files_list[i]["file_location"], 'rb') as f:
                        self.upload_result = post(self.upload_url,
                                                      headers=self.upload_headers,
                                                      data=self.upload_data,
                                                      files={"videofile": (
                                                title, 
                                                f, 
                                                self.files_list[i]["file_mimetype"])}
                                                        )
                    if self.upload_result.json()["video"]:
                        logging.info("Finished upload successfully")
                        self.files_list[i]["status"] = "completed"
                        
                        if self.upload_result.json()["video"]["uuid"]:
                            self.files_list[i]["uuid"] = self.upload_result.json()["video"]["uuid"]
                            logging.info("{}".format(self.files_list[i]["uuid"]))
                            
                            logging.info("Adding Metadata to video")
                            logging.info("tags: {}".format(self.files_list[i]["tags"]))
                            logging.info("category: {}".format(self.config["category"]))
                            logging.info("licence: {}".format(self.config["licence"]))
                            logging.info("language: {}".format(self.config["language"]))
                            logging.info("privacy: {}".format(self.config["privacy"]))
                            logging.info("nsfw: {}".format(self.config["nsfw"]))
                            logging.info("commentsEnabled: {}".format(self.config["commentsEnabled"]))
                            logging.info("downloadEnabled: {}".format(self.config["downloadEnabled"]))
                            logging.info("support: {}".format(self.config["support"]))
                            
                            res = self.set_video_metadata(self.files_list[i]["uuid"],{
                                "tags":self.files_list[i]["tags"],
                                "category":self.config["category"],
                                "licence":self.config["licence"],
                                "language":self.config["language"],
                                "privacy":self.config["privacy"],
                                "nsfw":self.config["nsfw"],
                                "commentsEnabled":self.config["commentsEnabled"],
                                "downloadEnabled":self.config["downloadEnabled"],
                                "support":self.config["support"]
                            })
                            logging.info("upload result: {}".format(res.content))
                            
                        
                        self.write_csv(self.csv_file,self.files_list)
                        
                    else:
                        if self.upload_result:
                            logging.error("Aborting upload. Error: {}".format(
                                                        self.upload_result.text
                                                        ))
                            self.files_list[i]["status"] = "error"
                        else:
                            logging.error("Aborting upload")
                            self.files_list[i]["status"] = "error"

                except Exception as e:
                    logging.error(e)
                    self.files_list[i]["status"] = "error"
                    if self.upload_result:
                            logging.error("Aborting upload. Error: {}".format(
                                                        self.upload_result.text
                                                        ))
                    else:
                            logging.error("Aborting upload")




In [ ]:
app = pypeertube()

In [ ]:
app.load_csv()

In [ ]:
app.upload()